In [1]:
import os
import sys
import re
import json
from pprint import pprint
from dotenv import load_dotenv
import google.generativeai as palm

sys.path.append('..')
from backend.DB.db import Database

In [2]:
load_dotenv()
api_key = os.getenv('PALM_API_KEY')
palm.configure(api_key=api_key)

In [3]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [4]:
db = Database()

sql = '''
SELECT topic_name FROM topics
WHERE category = "hotel";
'''

topics = db.query_sql(sql)
topics = [item[0] for item in topics]

with open(os.path.join('backend', 'gpt-instructions.txt'), 'r') as FILE:
    instructions = FILE.read()

instructions = instructions.replace('%%topic_list%%', str(topics))
print(instructions)

Your task is to analyze and summarize salient points under each topic from
a review. These are your tasks:

* Indicate if a topic is mentioned in the review: true or false

* Evaluate if the guest was angry about that aspect of their stay:  true or false.

* Evaluate the sentiment on a scale of 1 to 5. Scale is:

0 - Not applicable
1 - very negative
2 - negative
3 - neutral
4 - positive
5 - very positive

* Summarize the guest's comments on the topic, in 2 - 3 sentences.

Analyze the review below as instructed above, delimited by triple 
ticks. Format as JSON, exactly as follows. Include all topics and key/value pairs.

{topic : {"Relevant": value, "Angry": value, "Sentiment": value, "Summary": summary}}

Topics = ['amenities', 'frontdesk service', 'hotel condition', 'housekeeping', 'value']



In [6]:
results = db.query('raw_reviews')
len(results)

23

In [8]:

review = f"{results[22]['title']}. {results[22]['review_text']}"
pprint(review)
print('========================\n\n')
prompt = f'{instructions}\n\n=====================\n\n{review}'


completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0
)

read_review = completion.result
read_review = read_review.replace('\'', '\"')
read_review = read_review.replace('\n', '')
read_review = read_review.replace('True', 'true')
read_review = read_review.replace('False', 'false')

pprint(json.loads(read_review))

('Friendly and Clean Boutique Hotel. For a one-night stay when driving '
 'through, this was OK. The staff was super friendly, and it was clean, though '
 'small, and the property is off the main street, which was a plus. Moab is '
 'generally very walkable, and the Gonzo Inn was a good stop after a day of '
 'driving. The room was small, and we were given a room with a window next to '
 'the headboard, which also overlooked the parking lot. There was the '
 'potential for a noisy night, but it was okay. There were some loud talkers '
 'outside, and after turning out the light and some groaning, it was quiet, '
 "and we got a good night's sleep. Did not try the free breakfast, "
 'unfortunately. Overall, a bit pricey for one night, but it was clean, which '
 'is why I chose it. ')


{'amenities': {'Angry': False,
               'Relevant': True,
               'Sentiment': 3,
               'Summary': 'The hotel has free breakfast, but the guest did not '
                          'try